In [1]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import os
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

class Reporting():
    def __init__(self, gte, lte, view_id):
            self.SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
            self.KEY_FILE_LOCATION = '.KEY_FILE_LOCATION'
            self.SERVICE_ACCOUNT_EMAIL = 'SERVICE_ACCOUNT_EMAIL'
            self.GA_VIEW_ID = view_id
            self.gte = gte
            self.lte = lte
            self.analytics = self.initialize_analyticsreporting()

    def initialize_analyticsreporting(self):
        credentials = ServiceAccountCredentials.from_json_keyfile_name(self.KEY_FILE_LOCATION, self.SCOPES)
        # Build the service object.
        analytics = build('analyticsreporting', 'v4', credentials=credentials)
        return analytics
    
    
    def get_report(self, analytics):
        return analytics.reports().batchGet(
            body={
                'reportRequests' : [
                    {
                        'viewId' : self.GA_VIEW_ID,
                        #'dateRanges' : [{'ga:yearWeek':self.YearWeek}],
                        'dateRanges' : [{'startDate': self.gte, 'endDate': self.lte}],
                        'dimensions' : [{'name' : "ga:productSku"}],
                        "metrics" : [{"expression" : "ga:productDetailViews"},
                                     {"expression" : "ga:itemQuantity"}],
                        "pageSize" : 10000,
                    }]
            }
        ).execute()


def ga2df(gte, lte, view_id):
    data_list =[]
    r = Reporting(gte, lte, view_id)
    analytics = r.analytics
    response = r.get_report(analytics)
    results = response['reports'][0]['data']['rows']
    for i in range(len(results)):
        data = {}
        result = results[i]
        data['page_id'] = result['dimensions'][0]
        data['sessions'] = result['metrics'][0]['values'][0]
        data['sales'] = result['metrics'][0]['values'][1]
        data_list.append(data)
    
    df = pd.DataFrame()
    for i in range(len(data_list)):
        data = data_list[i]
        data = json_normalize(data)
        df  = df.append(data, ignore_index=True)
    df = df[['page_id', 'sessions', 'sales']]
    df['sessions'] = df['sessions'].astype(int)
    df['sales'] = df['sales'].astype(int)
    df['cov_ratio'] = np.round(df['sales'] / df['sessions'], 4)
    
    return df

In [2]:
ga2df('2017-11-19', '2017-11-25', 'view_id').sample(2)

,page_id,sessions,sales,cov_ratio
103,1938252,2,0,0.0
824,3597106,3,0,0.0
